In [78]:
!python -m pip install --user --upgrade pip

In [79]:
!pip3 install kfp --upgrade --user

In [89]:
import kfp
from kfp import dsl
import kfp.components as comp

In [100]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavendevv/load_airline:v.0.2',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [101]:
def stat_op(data):
  return dsl.ContainerOp(
        name = 'Dataset Statistics',
        image = 'mavendevv/stats:v.0.2',
        arguments = ['--data', data],
        file_outputs={
            'stats': '/statgen/stats',
            'mlpipeline-ui-metadata': '/mlpipeline-ui-metadata.json'
        }
    )

In [102]:
def schema_op(stats):
  return dsl.ContainerOp(
        name = 'Dataset Schema',
        image = 'mavencodevv/schema_heart:v.0.2',
        arguments = ['--stats', stats],
        file_outputs={
            'schema': '/schema/schema',
            'mlpipeline-ui-metadata': '/mlpipeline-ui-metadata.json'
        }      
    )

In [103]:
def val_op(stats, schema):
  return dsl.ContainerOp(
        name = 'Dataset Validation',
        image = 'mavendevv/validate:v.0.2',
        arguments = ['--stats', stats, '--schema', schema],
        file_outputs={
            'mlpipeline-ui-metadata': '/mlpipeline-ui-metadata.json'}
    )

In [104]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Feature Engineering',
        image = 'mavendevv/preprocess_airline2:v.0.2',#'mavencodevv/preprocess_heart:v.0.1',
        arguments = ['--data', data],
        file_outputs={
            'clean_data': '/preprocess/clean_data'   
        }
    )

In [105]:
@dsl.pipeline(
    name='Feature Engineering Pipeline',
   description='An ML reusable pipeline that performs feature engineering on data'
)

# Define parameters to be fed into pipeline
def fe_pipeline(): 
    
    _load_data_op = load_data_op()
    
    _stat_op  = stat_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    _stat_op.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    _schema_op = schema_op(
      dsl.InputArgumentPath(_stat_op.outputs['stats'])).after(_stat_op)
    _schema_op.execution_options.caching_strategy.max_cache_staleness = "P0D"
    _val_op = val_op(
      dsl.InputArgumentPath(_stat_op.outputs['stats']),
      dsl.InputArgumentPath(_schema_op.outputs['schema'])).after(_stat_op,_schema_op)
    
    _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op,_val_op)

In [106]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'fe_pipeline'

kfp.compiler.Compiler().compile(fe_pipeline, '{}.yaml'.format(experiment_name))

In [97]:
client = kfp.Client()
#client.create_run_from_pipeline_func(fe_pipeline, arguments={})

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').
